In [1]:
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py

--2023-08-06 23:19:39--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      82  --.-KB/s    in 0s      

2023-08-06 23:19:39 (7.38 MB/s) - ‘requirements.txt’ saved [82/82]

--2023-08-06 23:19:39--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2

In [2]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

In [3]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

In [4]:
set_seed(train_config.seed, reproducible=True)

In [5]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)

wandb: Currently logged in as: yihuanghz95. Use `wandb login --relogin` to force relogin


In [6]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 846.07MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:12.7


In [7]:
df.head()

,File_Name,Stage
0,24f4917f-93b8700a.jpg,train
1,128094d1-7314f248.jpg,train
2,520180a2-9a74b4a5.jpg,test
3,99ab7304-32307eea.jpg,train
4,c2ccc2a6-300804e4.jpg,test


In [8]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [9]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [10]:
# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [11]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [12]:
#training config
config = wandb.config

In [13]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

In [14]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/yi/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [15]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

In [16]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.511471,0.381218,0.300047,0.839934,0.694734,0.000000,0.000000,0.000000,0.565662,0.000000,00:20
1,0.409381,0.301297,0.329065,0.889863,0.790007,0.000000,0.000000,0.000000,0.623582,0.000000,00:20
2,0.367911,0.402031,0.296308,0.843305,0.583752,0.000000,0.000000,0.000000,0.647098,0.000000,00:20
3,0.304190,0.271115,0.334386,0.894662,0.801099,0.000000,0.000000,0.000000,0.644939,0.000000,00:20
4,0.276597,0.265999,0.349844,0.904110,0.821454,0.023863,0.000000,0.000000,0.699479,0.000000,00:20
5,0.255362,0.249475,0.351039,0.908859,0.822797,0.025245,0.000000,0.000000,0.700370,0.000000,00:20
6,0.224501,0.235783,0.372809,0.914945,0.831665,0.117226,0.000000,0.000000,0.745827,0.000000,00:20
7,0.203293,0.229380,0.374844,0.919486,0.842103,0.104404,0.002008,0.000000,0.755906,0.000000,00:20
8,0.192273,0.231374,0.379129,0.919899,0.839328,0.129728,0.000337,0.000000,0.764612,0.000000,00:20
9,0.177743,0.222733,0.379408,0.922857,0.845443,0.116938,0.000539,0.000000,0.770082,0.000000,00:20


Better model found at epoch 0 with miou value: 0.3000471158256307.
Better model found at epoch 1 with miou value: 0.32906461043170004.
Better model found at epoch 3 with miou value: 0.33438579424478293.
Better model found at epoch 4 with miou value: 0.34984380751070987.
Better model found at epoch 5 with miou value: 0.3510388027211332.
Better model found at epoch 6 with miou value: 0.3728088940168706.
Better model found at epoch 7 with miou value: 0.37484385757239586.
Better model found at epoch 8 with miou value: 0.3791292982050898.
Better model found at epoch 9 with miou value: 0.3794084577395318.


In [17]:
# log prediction
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

In [18]:
# load to summary
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [19]:
wandb.finish()


background_iou,▁▅▁▆▆▇▇███
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▁▄▁▄▆▆▇███
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
